# Data Analysis

In [1]:
import pandas as pd 
import glob
import re
import sys
import concurrent.futures
from Analysis import Analysis

pd.set_option('display.max_rows', 100)

### Calculate the result for each bug

In [2]:
root = "/home/jovyan/work"
analyzer = Analysis(root)

projects = [
    "Time", "JacksonXml", "Collections", "Compress", "Csv", "JacksonCore", "JacksonDatabind", 
    "Jsoup", "Lang", "Math","Gson", "Closure", "Mockito"
]
future_results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    for project in projects:
        # FOR EACH BUG
        for bug_path in glob.glob("{root}/results/{project}/Bug_*/".format(root=root, project=project)):
            bug = re.search(r"Bug_(\d+)", bug_path).group(1)
            try:
                future = executor.submit(analyzer.analyzeBug, project, bug, False)
                future_results.append(future)
            except Exception as e:
                print(bug_path)
                print(e)

In [3]:
bug_results = []
for future in future_results:
    r = future.result() 
    bug_results.append(r)
#     try:
        
#     except Exception as e:
#         print(e)

### Example of bug result when a BIC detected

In [4]:
bug_results[9]

{'id': 'Time_Bug_4',
 'bug': 'Bug_4',
 'project': 'Time',
 'fix_pass': True,
 'prev_fails': True,
 'category': 'No regression is detected',
 'sub_category': '-',
 'test_name': 'org.joda.time.TestPartial_Basics#testWith3',
 'bug_report': 'https://github.com/JodaOrg/joda-time/issues/88',
 'fix_commit': '3ba9ba799b3261b7332a467a88be142c83b298fd',
 'BIC_candidates': [],
 'executionsOnPast': 148,
 'buildFail': 0,
 'buildTestFail': 1493,
 'numCommits': 1714}

## RQ0: “Can regression tests be transplanted to the past?”

In [5]:
resume_df= pd.DataFrame(bug_results)[
    ['id', 'project','executionsOnPast', 'numCommits', 'buildFail', 'buildTestFail', 'BIC_candidates']
]

# Remove bugs with non-succeed BFC
resume_df = resume_df[resume_df['numCommits'] > 1]

resume_df['one_bic_found'] = resume_df.apply (
    lambda row: 1 if len(row['BIC_candidates']) == 1 else 0, axis=1
)
resume_df['n_bic_found'] = resume_df.apply (
    lambda row: 1 if len(row['BIC_candidates']) > 1 else 0, axis=1
)
resume_df['executionsOnPast_rate'] = resume_df.apply (
    lambda row: row['executionsOnPast'] * 100 / row['numCommits'], axis=1
)
resume_df['compilability'] = resume_df.apply (
    lambda row: (row['numCommits']-row['buildFail']) * 100 / row['numCommits'], axis=1
)
resume_df['testCompilability'] = resume_df.apply (
    lambda row: (row['numCommits']-row['buildTestFail']-row['buildFail']) * 100 / row['numCommits'], axis=1
)


resume_df.groupby('project').agg(
    bugs=('project','size'), 
    #s_bics=('one_bic_found','sum'),
    #m_bics=('n_bic_found','sum'),
    compilability_mean=('compilability','mean'), 
    compilability_median=('compilability','median'),
    test_compilability_mean=('testCompilability','mean'), 
    test_compilability_median=('testCompilability','median'),
    ratio_mean=('executionsOnPast_rate','mean'), 
    ratio_median=('executionsOnPast_rate','median'),
    #commits_mean=('numCommits','mean'),
    #commits_median=('numCommits','median')
).round(2)

,bugs,compilability_mean,compilability_median,test_compilability_mean,test_compilability_median,ratio_mean,ratio_median
project,,,,,,,
Closure,147,61.10,53.37,30.70,15.40,30.70,15.40
Collections,2,95.94,95.94,3.99,3.99,3.77,3.77
Compress,40,26.11,21.21,19.43,14.63,16.55,8.16
Csv,15,18.95,16.56,5.09,3.44,5.03,3.44
Gson,18,42.00,35.98,40.73,34.68,40.73,34.68
JacksonCore,23,40.48,38.45,36.72,30.50,36.72,30.50
JacksonDatabind,97,90.16,92.80,15.78,16.01,15.78,16.01
JacksonXml,3,85.02,84.39,37.66,36.21,37.66,36.21
Jsoup,91,20.83,12.43,17.83,10.00,17.83,10.00


In [6]:
resume_df['executionsOnPast_rate'].mean()

19.776225404212212

In [7]:
resume_df['executionsOnPast_rate'].median()

11.616472034419177

## RQ1: “Can the BIC for a given bug be found using its regression test?”

### Summary of results

In [8]:
df = pd.DataFrame(bug_results)[['id', 'project','category', 'sub_category']]
df[df['category'] != None].set_index('id').sort_index()
print(len(df))
df['category'].value_counts()

730


No regression is detected       580
A regression is detected         77
Test fails in the fix commit     73
Name: category, dtype: int64

### Results per sub-catergory

In [9]:
df.groupby(['category', "sub_category"]).count()['id']

category                      sub_category             
A regression is detected      Multiple candidates           27
                              Unique candidates             50
No regression is detected     -                            580
Test fails in the fix commit  Failure in source build       52
                              Failure in test build          3
                              Test execution fails           2
                              The test was not executed     16
Name: id, dtype: int64

### Test fails in the fix commit - Test execution fails

In [10]:
df[df['sub_category']=='Test execution fails']

,id,project,category,sub_category
239,Jsoup_Bug_67,Jsoup,Test fails in the fix commit,Test execution fails
316,Jsoup_Bug_78,Jsoup,Test fails in the fix commit,Test execution fails
